<a href="https://colab.research.google.com/github/Soumya1029/First_project/blob/main/Object_Detection_and_Tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow keras opencv-python numpy


In [16]:
!pip install opencv-python


In [ ]:
!wget -q https://pjreddie.com/media/files/yolov3.weights
!wget -q https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
!wget -q https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving traffic-mini.mp4 to traffic-mini.mp4


In [8]:
import cv2
import numpy as np
import os

# # Download YOLO weights, config, and coco names
# !wget -q https://pjreddie.com/media/files/yolov3.weights
# !wget -q https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
# !wget -q https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names

# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load class names
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Generate colors for each class
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Upload the video file
from google.colab import files
uploaded = files.upload()

# Extract the file name
video_file = list(uploaded.keys())[0]

# Initialize video capture with the uploaded video file
cap = cv2.VideoCapture(video_file)

# Check if video capture is initialized successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
else:
    # Get the width and height of the frames
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f"Frame Width: {frame_width}, Frame Height: {frame_height}")

    # Temporary directory to store frames
    temp_dir = './frames'
    os.makedirs(temp_dir, exist_ok=True)

    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        height, width, channels = frame.shape

        # Prepare the frame for the model
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        # Process the outputs for YOLO
        class_ids = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                color = colors[class_ids[i]]
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(frame, label, (x, y + 30), cv2.FONT_HERSHEY_PLAIN, 3, color, 3)

        # Save frame as image in temporary directory
        frame_count += 1
        frame_filename = os.path.join(temp_dir, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(frame_filename, frame)

        # Optionally display the frame (uncomment if you want to see the frames)
        # cv2_imshow(frame)

        # Add a break condition: stop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # Convert frames to video using ffmpeg
    output_video = 'output.mp4'
    ffmpeg_cmd = f"ffmpeg -framerate 30 -i {temp_dir}/frame_%04d.jpg -c:v libx264 -pix_fmt yuv420p {output_video}"
    os.system(ffmpeg_cmd)

    print(f"Video processing completed and saved as '{output_video}'.")

    # Download the video
    from google.colab import files
    files.download(output_video)

    # Clean up: delete temporary directory
    import shutil
    shutil.rmtree(temp_dir)


Saving traffic-mini.mp4 to traffic-mini (6).mp4
Frame Width: 854, Frame Height: 480
Video processing completed and saved as 'output.mp4'.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>